In [378]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
#/Users/shuxianglei/Desktop/Practicum/code/practicum/JSON_Data/Hexagon_GMRF_parsed_text.json

data=pd.read_excel("/Users/shuxianglei/Desktop/FIN 554/SP500.xlsx")
# data=pd.read_excel("D:\\stock_data\\mapping_merge.xlsx")
data

,Date,Open,High,Low,Close,Adj_Close,Log_price
0,1985-01-01,167.20,180.27,163.36,179.63,179.63,0.008592
1,1985-02-01,179.63,183.95,177.75,181.18,181.18,-0.002874
2,1985-03-01,181.18,183.89,176.53,180.66,180.66,-0.004605
3,1985-04-01,180.66,183.61,177.86,179.83,179.83,0.052641
4,1985-04-30,179.83,189.98,178.35,189.55,189.55,0.012061
...,...,...,...,...,...,...,...
453,2022-09-30,3609.78,3905.42,3491.58,3871.98,3871.98,0.052358
454,2022-10-31,3901.79,4080.11,3698.15,4080.11,4080.11,-0.060782
455,2022-12-01,4087.14,4100.96,3764.49,3839.50,3839.50,0.059921
456,2023-01-01,3853.29,4094.21,3794.33,4076.60,4076.60,-0.026487


In [483]:
import yfinance as yf
import pandas as pd


sp500 = yf.Ticker("^GSPC")
history = sp500.history(period="max")
history.index = pd.to_datetime(history.index)
last_day_prices = history.resample("M").last()["Close"]
monthly_data_SP500 = pd.DataFrame({
    "Close": last_day_prices
})
monthly_data_SP500['Pct_Change'] = monthly_data_SP500['Close'].pct_change()
monthly_data_SP500



,Close,Pct_Change
Date,,
1927-12-31 00:00:00-05:00,17.660000,NaN
1928-01-31 00:00:00-05:00,17.570000,-0.005096
1928-02-29 00:00:00-05:00,17.260000,-0.017644
1928-03-31 00:00:00-05:00,19.280001,0.117034
1928-04-30 00:00:00-04:00,19.750000,0.024378
...,...,...
2022-11-30 00:00:00-05:00,4080.110107,0.053753
2022-12-31 00:00:00-05:00,3839.500000,-0.058971
2023-01-31 00:00:00-05:00,4076.600098,0.061753


In [490]:
price = monthly_data_SP500["Pct_Change"].values[1:]
miu = price.mean()
sigma = np.sqrt(price.var())
print(miu)
print(sigma)
u=miu+sigma*0.35
d=miu-sigma*0.48
print(u)
print(d)

0.0062157095182418555
0.05375736961201765
0.025030788882448033
-0.019587827895526617


In [489]:
import math
from datetime import timedelta
import json
result=[]
signal =0
for i in range(0, len(monthly_data_SP500["Pct_Change"])-2):
    if monthly_data_SP500.iloc[i+1, 1] < d and monthly_data_SP500.iloc[i+2, 1] < d:
        # if monthly_data_SP500.iloc[i+2, 1] > 0.05:
        # result.append([monthly_data_SP500.index[i],-1])
        if signal==0:
            signal=-1
            result.append([monthly_data_SP500.index[i],-1])
        elif signal==-1:
            pass
        elif signal==1:
            signal=-1
            result.append([monthly_data_SP500.index[i],-1])
    
    elif monthly_data_SP500.iloc[i+1, 1] > u and monthly_data_SP500.iloc[i+2, 1] > u:
        if signal==0:
            signal=1
            result.append([monthly_data_SP500.index[i],1])
        elif signal==1:
            pass
        elif signal==-1:
            signal=1
            result.append([monthly_data_SP500.index[i],1])

result
bull_time=[]
bear_time=[]
for i in range(len(result)-1):
    if result[i][1]==1:
        bull_time.append(round((result[i+1][0]-result[i][0]).days / 30, 3))
    elif result[i][1]==-1:
        bear_time.append(round((result[i+1][0]-result[i][0]).days / 30, 3))
if result[-1][1]==1:
    bull_time.append(round((monthly_data_SP500.index[-1]-result[-1][0]).days / 30, 3))

elif result[-1][1]==-1:
    bear_time.append(round((monthly_data_SP500.index[-1]-result[-1][0]).days / 30, 3))
print(bull_time)
print(bear_time)

print(len(bull_time))
print(sum(bull_time) / len(bull_time))
print(len(bear_time))
print(sum(bear_time) / len(bear_time))
bb_dict = {"bull":bull_time,"bear":bear_time}
json_str = json.dumps(bb_dict)
with open("sp500_bb.json","w",encoding="utf-8") as json_file:
    json_file.write(json_str)
    json_file.close()



[11.167, 9.1, 1.967, 2.067, 6.1, 28.4, 40.6, 45.633, 7.133, 13.2, 13.133, 20.3, 29.433, 4.033, 49.733, 58.9, 44.633, 6.0, 29.467, 7.067, 59.867, 32.433, 37.567, 67.0, 2.033, 57.867, 14.167, 10.1, 43.567, 19.333, 5.1, 14.2, 6.067]
[3.033, 4.067, 16.233, 6.033, 19.233, 10.133, 11.133, 12.167, 2.0, 12.167, 2.033, 12.167, 8.1, 7.067, 11.133, 30.367, 10.2, 36.533, 10.1, 5.1, 3.033, 5.1, 4.967, 33.433, 4.033, 14.167, 4.1, 6.133, 35.5, 1.967, 2.033, 9.067]
33
24.162636363636366
32
11.016625


In [493]:
a = (33 * 24.162636363636366) / (33 * 24.162636363636366 + 32 * 11.016625)
b = (32 * 11.016625) / (33 * 24.162636363636366 + 32 * 11.016625)
print(a, b)

0.6934235093690837 0.30657649063091624


In [509]:
c = (33 * 24.162636363636366 + 32 * 11.016625)
c = c * 0.0689
c

79.22804110000001

In [510]:
import math
from datetime import timedelta
import json
result=[]
signal =0
countbear = 0
countbull = 0
for i in range(0, len(monthly_data_SP500["Pct_Change"])-2):
    if monthly_data_SP500.iloc[i+1, 1] < d:
        countbull += 1
        if signal==0:
            signal=-1
            result.append([monthly_data_SP500.index[i],-1])
        elif signal==-1:
            pass
        elif signal==1:
            signal=-1
            result.append([monthly_data_SP500.index[i],-1])
    
    elif monthly_data_SP500.iloc[i+1, 1] > u and monthly_data_SP500.iloc[i+2, 1] > u:
        countbear += 1
        if signal==0:
            signal=1
            result.append([monthly_data_SP500.index[i],1])
        elif signal==1:
            pass
        elif signal==-1:
            signal=1
            result.append([monthly_data_SP500.index[i],1])
print(countbull, countbear)

result
bull_time=[]
bear_time=[]
for i in range(len(result)-1):
    if result[i][1]==1:
        bull_time.append(round((result[i+1][0]-result[i][0]).days / 30, 3))
    elif result[i][1]==-1:
        bear_time.append(round((result[i+1][0]-result[i][0]).days / 30, 3))
if result[-1][1]==1:
    bull_time.append(round((monthly_data_SP500.index[-1]-result[-1][0]).days / 30, 3))

elif result[-1][1]==-1:
    bear_time.append(round((monthly_data_SP500.index[-1]-result[-1][0]).days / 30, 3))
print(bull_time)
print(bear_time)

# print(len(bull_time))
# print(sum(bull_time) / len(bull_time))
# print(len(bear_time))
# print(sum(bear_time) / len(bear_time))
# bb_dict = {"bull":bull_time,"bear":bear_time}
# json_str = json.dumps(bb_dict)
# with open("sp500_bb.json","w",encoding="utf-8") as json_file:
#     json_file.write(json_str)
#     json_file.close()



284 122
[7.067, 3.067, 6.033, 1.967, 2.067, 4.033, 12.2, 11.167, 2.033, 5.1, 4.067, 10.1, 2.033, 6.133, 3.033, 2.033, 4.033, 2.0, 9.1, 7.167, 4.0, 5.067, 11.1, 2.967, 2.033, 4.033, 18.267, 7.033, 3.067, 3.033, 6.033, 2.033, 10.133, 6.0, 3.067, 8.133, 2.033, 9.1, 9.1, 8.067, 3.033, 5.1, 6.067, 5.0, 2.067, 17.167, 3.033, 4.033, 6.0, 5.1, 2.0, 2.033, 15.2, 2.033, 3.033, 7.1, 2.933, 10.1, 4.0, 4.067, 3.067, 3.967, 3.067, 5.1, 10.133, 2.033]
[4.067, 1.033, 3.033, 7.133, 16.233, 6.033, 21.3, 0.967, 14.167, 22.333, 7.033, 11.133, 10.167, 13.2, 0.933, 12.167, 7.1, 21.3, 2.033, 2.033, 9.133, 12.167, 3.067, 4.1, 1.033, 8.1, 7.067, 14.233, 4.067, 14.133, 27.433, 12.167, 38.533, 44.7, 36.533, 18.267, 10.1, 10.1, 27.433, 6.133, 3.033, 6.1, 1.967, 15.233, 37.533, 5.1, 2.067, 4.033, 6.133, 1.033, 0.933, 41.633, 4.033, 4.1, 27.3, 20.3, 4.1, 4.1, 6.133, 16.267, 54.7, 3.067, 6.133, 1.967, 2.033, 13.167, 4.0]


In [504]:
from scipy.stats import binned_statistic_2d
sp500_returns = (monthly_data_SP500['Close'].pct_change()+1).fillna(1).values
sp500_states = np.zeros_like(sp500_returns)
sp500_states[sp500_returns > 1] = 1
transition_matrix, _, _ = binned_statistic_2d(sp500_states[:-1], sp500_states[:1], sp500_states[1:], sp500_returns[1:], statistic='count', bins=[2, 2])
transition_matrix = transition_matrix / np.sum(transition_matrix, axis=1, keepdims=True)
print(transition_matrix[0, 1])



TypeError: binned_statistic_2d() got multiple values for argument 'statistic'

DJA

In [482]:
import yfinance as yf
import pandas as pd


dji = yf.download("^DJA", start='1979-01-01', end='2023-03-03')
dji.drop[dji["Open", "High", "Low", "Close", "Volume"]]
dji
# history = dji.history(period="max")
# history.index = pd.to_datetime(history.index)
# last_day_prices = history.resample("M").last()["Close"]
# monthly_data_dji = pd.DataFrame({
#     "Close": last_day_prices
# })
# monthly_data_dji
# monthly_data_dji['Pct_Change'] = monthly_data_dji['Close'].pct_change()
# monthly_data_dji

[*********************100%***********************]  1 of 1 completed


KeyError: ('Open', 'High', 'Low', 'Close', 'Volume')

In [432]:
price = monthly_data_dji["Pct_Change"].values[1:]
miu = price.mean()
sigma = np.sqrt(price.var())
print(miu)
print(sigma)
u=miu+sigma*0.09
d=miu-sigma*0.09

0.0071731369570326355
0.04229276357221011


In [465]:
price = monthly_data_dji["Pct_Change"].values[1:]
miu = price.mean()
sigma = np.sqrt(price.var())
# print(miu)
# print(sigma)
u=miu+sigma*0.9
d=miu-sigma*0.9
import math
from datetime import timedelta
import json
result=[]
signal =0

for i in range(0, len(monthly_data_dji["Pct_Change"])-2):
    if monthly_data_dji.iloc[i+1, 1] < d:
        # if monthly_data_SP500.iloc[i+2, 1] > 0.05:
        # result.append([monthly_data_SP500.index[i],-1])
        if signal==0:
            signal=-1
            result.append([monthly_data_dji.index[i],-1])
        elif signal==-1:
            pass
        elif signal==1:
            signal=-1
            result.append([monthly_data_dji.index[i],-1])
    
    elif monthly_data_dji.iloc[i+1, 1] > u and monthly_data_dji.iloc[i+2, 1] > u and monthly_data_dji.iloc[i+3, 1] > u:
        if signal==0:
            signal=1
            result.append([monthly_data_dji.index[i],1])
        elif signal==1:
            pass
        elif signal==-1:
            signal=1
            result.append([monthly_data_dji.index[i],1])

result
bull_time=[]
bear_time=[]
for i in range(len(result)-1):
    if result[i][1]==1:
        bull_time.append(round((result[i+1][0]-result[i][0]).days / 30, 3))
    elif result[i][1]==-1:
        bear_time.append(round((result[i+1][0]-result[i][0]).days / 30, 3))
if result[-1][1]==1:
    bull_time.append(round((monthly_data_dji.index[-1]-result[-1][0]).days / 30, 3))

elif result[-1][1]==-1:
    bear_time.append(round((monthly_data_dji.index[-1]-result[-1][0]).days / 30, 3))
# print(bull_time)
# print(bear_time)

print(len(bull_time))
print(sum(bull_time) / len(bull_time))
print(len(bear_time))
print(sum(bear_time) / len(bear_time))
# bb_dict = {"bull":bull_time,"bear":bear_time}
# json_str = json.dumps(bb_dict)
# with open("dji_bb.json","w",encoding="utf-8") as json_file:
#     json_file.write(json_str)
#     json_file.close()



1
4.033
2
184.6335


In [308]:
# list = [1.2, 1.2, 0.8, 0.7, 0.9, 1.2, 1.5, 1.7, 1.8, 1.7]

# result = []

# for i in range(0, len(list)-1):
#     if list[i+1] - list[i] < 0:
#         result.append([i, i+1])
# final_result = [result[0]]

# for i in range(1, len(result)):
#     comparison = final_result[-1]
#     if comparison[1] == result[i][0]:
#         final_result.pop()
#         final_result.append([comparison[0], result[i][1]])
#     else:
#         final_result.append(result[i])

# final_result
lst = [[4, 5], [3, 6]]

# 嵌套循环和列表推导式求每个小list的差值
diffs = [b - a for a, b in lst]

print(diffs) 

[1, 3]
